- https://www.databricks.com/blog/introducing-english-new-programming-language-apache-spark
- https://github.com/databrickslabs/pyspark-ai/

In [0]:
from pyspark_ai import SparkAI
# langchain is auto installed as a dependency of pyspark-ai
from langchain.chat_models import ChatOpenAI
from pyspark.sql import functions as fn, types as T

In [0]:
# If 'gpt-4' is unavailable, use 'gpt-3.5-turbo' (might lower output quality)
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Initialize SparkAI with the ChatOpenAI model
spark_ai = SparkAI(llm=llm, verbose=True)

In [0]:
spark_ai.activate()

In [0]:
# a super useful help prompt
help(spark_ai)

Help on SparkAI in module pyspark_ai.pyspark_ai object:

class SparkAI(builtins.object)
 |  SparkAI(llm: Optional[langchain.base_language.BaseLanguageModel] = None, web_search_tool: Optional[Callable[[str], str]] = None, spark_session: Optional[pyspark.sql.session.SparkSession] = None, enable_cache: bool = True, cache_file_format: str = 'json', cache_file_location: Optional[str] = None, encoding: Optional[tiktoken.core.Encoding] = None, max_tokens_of_web_content: int = 3000, verbose: bool = False) -> None
 |  
 |  Methods defined here:
 |  
 |  __init__(self, llm: Optional[langchain.base_language.BaseLanguageModel] = None, web_search_tool: Optional[Callable[[str], str]] = None, spark_session: Optional[pyspark.sql.session.SparkSession] = None, enable_cache: bool = True, cache_file_format: str = 'json', cache_file_location: Optional[str] = None, encoding: Optional[tiktoken.core.Encoding] = None, max_tokens_of_web_content: int = 3000, verbose: bool = False) -> None
 |      Initialize the 

# Plotting Data

This can be done either by passing a dataframe, or by also providing some instructions to indicate the type of plot required. Note that usefully, a wireframe design of how the data was prepared for plotting is provided

In [0]:
# lets use a builtin databricks dataset
display(dbutils.fs.ls('/databricks-datasets/wine-quality/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/wine-quality/README.md,README.md,1066,1594264539000
dbfs:/databricks-datasets/wine-quality/winequality-red.csv,winequality-red.csv,84199,1594264539000
dbfs:/databricks-datasets/wine-quality/winequality-white.csv,winequality-white.csv,264426,1594264539000


In [0]:
df_wine = (spark.read
  .format("csv")
  .option("header", "true")
  # this data has an unusual delimiter
  .option("delimiter", ";")
  .option("inferSchema", "true")
  .load("/databricks-datasets/wine-quality/winequality-red.csv")
)

In [0]:
df_wine.ai.plot()

INFO: To visualize the result of a Spark DataFrame using Plotly, we need to convert the DataFrame to a Pandas DataFrame first. However, since the requirement is to not use any deprecated APIs, we cannot use the `toPandas()` method.

Instead, we can use the `collect()` method of the DataFrame to retrieve all the rows as a list of `Row` objects. Then, we can convert this list of `Row` objects to a list of dictionaries, where each dictionary represents a row with column names as keys and corresponding values.

Once we have the list of dictionaries, we can create a Pandas DataFrame using the `pd.DataFrame()` constructor. Finally, we can use Plotly to visualize the Pandas DataFrame.

Here's the code to achieve this:


```
import pandas as pd
import plotly.express as px
from pyspark.sql import Row

# Convert each Row object to a dictionary
rows = df.collect()
data = [row.asDict() for row in rows]

# Create a Pandas DataFrame
pandas_df = pd.DataFrame(data)

# Visualize the Pandas DataFrame us

In [0]:
df_wine.ai.plot('plot total sulphur dioxide against pH')

INFO: To visualize the result of `df` using plotly, we can follow these steps:

1. Import the necessary libraries:

```
import plotly.express as px
import pyspark.sql.functions as F
```

2. Create a new dataframe `plot_df` by selecting the required columns from `df`:

```
plot_df = df.select("total sulfur dioxide", "pH")
```

3. Convert the Spark dataframe to a Pandas dataframe using `toPandas()` method:

```
pandas_df = plot_df.toPandas()
```

4. Use Plotly Express to create a scatter plot:

```
fig = px.scatter(pandas_df, x="total sulfur dioxide", y="pH")
```

5. Display the plot:

```
fig.show()
```

Here's the complete code:


```
import plotly.express as px
import pyspark.sql.functions as F

# Step 2
plot_df = df.select("total sulfur dioxide", "pH")

# Step 3
pandas_df = plot_df.toPandas()

# Step 4
fig = px.scatter(pandas_df, x="total sulfur dioxide", y="pH")

# Step 5
fig.show()
```

This code will plot the total sulfur dioxide against pH using Plotly Express.


# Transforming data

In [0]:
display(df_wine.limit(5))

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6
7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [0]:
display(df_wine.select('quality').distinct())

quality
6
3
5
4
8
7


In [0]:
df_averages = df_wine.ai.transform('group by pH and calculate the average chlorides and sulphate for each group, then order the results by ascending pH')

INFO: SQL query for the transform:
SELECT pH, AVG(chlorides) as avg_chlorides, AVG(sulphates) as avg_sulphates
FROM temp_view_for_transform
GROUP BY pH
ORDER BY pH ASC



In [0]:
display(df_averages)

pH,avg_chlorides,avg_sulphates
2.74,0.61,2.0
2.86,0.075,0.79
2.87,0.148,1.36
2.88,0.078,0.645
2.89,0.0775,0.47
2.9,0.414,1.33
2.92,0.08950000000000001,0.725
2.93,0.11,1.96
2.94,0.082,0.9800000000000001
2.95,0.1,0.68


In [0]:
# lets check the math
display(
  df_wine.groupBy(
    'pH'
  ).agg(
    fn.avg('chlorides'),
    fn.avg('sulphates')
  ).orderBy(
    fn.col('ph').asc()
  )
)
# looks like our AI calculated result is correct

pH,avg(chlorides),avg(sulphates)
2.74,0.61,2.0
2.86,0.075,0.79
2.87,0.148,1.36
2.88,0.078,0.645
2.89,0.0775,0.47
2.9,0.414,1.33
2.92,0.08950000000000001,0.725
2.93,0.11,1.96
2.94,0.082,0.9800000000000001
2.95,0.1,0.68


## Failure Case

The SQL generated fails to correctly handle a column name with a space in it

In [0]:
df_averages_2 = df_wine.ai.transform('group by "residual sugar" and calculate the average chlorides, sulphates and pH for each group')

INFO: SQL query for the transform:
SELECT 
  residual sugar,
  AVG(chlorides) AS avg_chlorides,
  AVG(sulphates) AS avg_sulphates,
  AVG(pH) AS avg_pH
FROM 
  temp_view_for_transform
GROUP BY 
  residual sugar



---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-3312646010236191>:1
----> 1 df_averages = df_wine.ai.transform('group by "residual sugar" and calculate the average chlorides, sulphates and pH for each group')

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/ai_utils.py:37, in AIMethodWrapper.transform(self, desc, cache)
     24 def transform(self, desc: str, cache: bool = True) -> DataFrame:
     25     """
     26     Transform the DataFrame using the given description.
     27 
   (...)
     35         The transformed DataFrame.
     36     """
---> 37     return self.spark_ai.transform_df(self.df_instance, desc, cache)

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:345, in SparkAI.transform_df(self, df, desc, cache)
    343 formatted_sql_query = CodeLog

# Creating UDFs

UDFS can be created with the spark_ai.udf decorator

In [0]:
@spark_ai.udf
def convert_quality(quality: int) -> str:
    """Convert the quality to a descriptive word where 1 would be awful and 10 would be amazing"""

INFO: Creating following Python UDF:
def convert_quality(quality) -> str:
    if quality is not None:
        if quality == 1:
            return "awful"
        elif quality == 2:
            return "bad"
        elif quality == 3:
            return "poor"
        elif quality == 4:
            return "fair"
        elif quality == 5:
            return "average"
        elif quality == 6:
            return "good"
        elif quality == 7:
            return "very good"
        elif quality == 8:
            return "excellent"
        elif quality == 9:
            return "outstanding"
        elif quality == 10:
            return "amazing"



In [0]:
df_wine.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



In [0]:
type(convert_quality)

function

In [0]:
convert_quality(2)

'bad'

In [0]:
# strangely the @spark_ai.udf decorator seems to produce a standard function rather than a udf function so the following fails
df_wine_descriptions = df_wine.withColumn(
  'description', convert_quality('quality')
)

In [0]:
# however that is easy to rectify by adding a udf wrapper to the function
df_wine_descriptions = df_wine.withColumn(
  'description', fn.udf(convert_quality)('quality')
)

In [0]:
display(df_wine_descriptions.select('quality', 'description'))

quality,description
5,average
5,average
5,average
6,good
5,average
5,average
5,average
7,very good
7,very good
5,average


# Reading data from the web 

This can be done either by providing a search term (if you have set GOOGLE_API_KEY in your environment variables) or by providing a hypertext link. Optionally you can provide a list of column names to assist the AI in parsing the data

In [0]:
df_auto = spark_ai.create_df("2022 USA national auto sales by brand")

---------------------------------------------------------------------------
ValidationError                           Traceback (most recent call last)
File <command-3312646010236185>:1
----> 1 auto_df = spark_ai.create_df("2022 USA national auto sales by brand")

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:293, in SparkAI.create_df(self, desc, columns, cache)
    291 # If the input is not a valid URL, use search tool to get the dataset.
    292 if not is_url:
--> 293     url = self._get_url_from_search_tool(desc, columns, cache)
    295 self.log(f"Parsing URL: {url}\n")
    296 try:

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:193, in SparkAI._get_url_from_search_tool(self, desc, columns, cache)
    190 def _get_url_from_search_tool(
    191     self, desc: str, columns: Optional[List[str]], cache: bool
    192 ) -> str:
--> 193     search_result = se

In [0]:
df_auto = spark_ai.create_df("https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand")

INFO: Parsing URL: https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW auto_sales AS 
SELECT 
    brand AS brand_name,
    sales_2022 AS sales_2022,
    sales_vs_2021 AS sales_vs_2021
FROM 
    (VALUES 
        ('Toyota', 1849751, -9),
        ('Ford', 1767439, -2),
        ('Chevrolet', 1502389, 6),
        ('Honda', 881201, -33),
        ('Hyundai', 724265, -2),
        ('Kia', 693549, -1),
        ('Jeep', 684612, -12),
        ('Nissan', 682731, -25),
        ('Subaru', 556581, -5),
        ('Ram Trucks', 545194, -16),
        ('GMC', 517649, 7),
        ('Mercedes-Benz', 350949, 7),
        ('BMW', 332388, -1),
        ('Volkswagen', 301069, -20),
        ('Mazda', 294908, -11),
        ('Lexus', 258704, -15),
        ('Dodge', 190793, -12),
        ('Audi', 186875, -5),
        ('Cadillac', 134726, 14),
        ('Chrysler', 112713, -2),
        ('Buick', 103519, -42),
        ('Acura', 102306, -35

In [0]:
display(df_auto)

brand_name,sales_2022,sales_vs_2021
Toyota,1849751,-9
Ford,1767439,-2
Chevrolet,1502389,6
Honda,881201,-33
Hyundai,724265,-2
Kia,693549,-1
Jeep,684612,-12
Nissan,682731,-25
Subaru,556581,-5
Ram Trucks,545194,-16


## Failure Cases

The AI is still quite brittle. As such there can be failure cases when trying to read from the web

In [0]:
# a failure case where the data is not recognised
# df_40K = spark_ai.create_df("https://40kstats.goonhammer.com/#home")

INFO: Parsing URL: https://40kstats.goonhammer.com/#home

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW warhammer_stats AS
SELECT 'Warhammer Stats Administratum TTBattles App 40k Stats Goonhammer Media The UTC Buttscribe Core Games Other Games Columns Gaming Tactics Reviews Hobby Site News Win Rate Win Rate and Points by Faction Win Rate by Faction/Mission Win Rate by Detachment Win Rate: Faction vs Faction Scoring Primary Scoring (All) Primary Scoring (Faction) Secondary Scoring (All) Secondary Scoring (Faction) Fixed vs Tactical Scoring Visualizations Faction Popularity Events Event Placings and Lists Top Finishes by Faction TiWP Report Faction Pages Mission Pack: Start Date: End Date: Follow us Facebook Instagram Twitter YouTube Are you a Goonhammer Patron? Click here to disable ads Manage your cookie settings 40kstats.com is a presentation of Goonhammer. Copyright Goonhammer, LLC Title Close' AS answer

INFO: Storing data into temp view: warhammer_stats



In [0]:
display(df_40K)

answer
"Warhammer Stats Administratum TTBattles App 40k Stats Goonhammer Media The UTC Buttscribe Core Games Other Games Columns Gaming Tactics Reviews Hobby Site News Win Rate Win Rate and Points by Faction Win Rate by Faction/Mission Win Rate by Detachment Win Rate: Faction vs Faction Scoring Primary Scoring (All) Primary Scoring (Faction) Secondary Scoring (All) Secondary Scoring (Faction) Fixed vs Tactical Scoring Visualizations Faction Popularity Events Event Placings and Lists Top Finishes by Faction TiWP Report Faction Pages Mission Pack: Start Date: End Date: Follow us Facebook Instagram Twitter YouTube Are you a Goonhammer Patron? Click here to disable ads Manage your cookie settings 40kstats.com is a presentation of Goonhammer. Copyright Goonhammer, LLC Title Close"


In [0]:
# a failure case where the data is not recognised even with column prompts
df_40K_rates = spark_ai.create_df(
  "https://40kstats.goonhammer.com/#GbF", 
  columns=["Faction",	"Games",	"VP",	"Opp VP",	"Win %",	"Wins",	"Losses",	"Draws",	"Real Win %"]
)

INFO: Parsing URL: https://40kstats.goonhammer.com/#GbF

INFO: SQL query for the ingestion:
Based on the given information, it seems that the required data is not available in the provided answer. The answer only includes information about various statistics and options related to Warhammer Stats, but it does not provide any actual data for the columns mentioned in the question.

To create a SQL query to store the answer into a temp view, we would need access to a database or table that contains the required data. Without the actual data, it is not possible to write a query to store it in a temp view.

If you have access to the actual data related to Warhammer Stats, please provide it so that a query can be written to store it in a temp view.



---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-3312646010236179>:1
----> 1 df_40K_rates = spark_ai.create_df(
      2   "https://40kstats.goonhammer.com/#GbF", 
      3   columns=["Faction",	"Games",	"VP",	"Opp VP",	"Win %",	"Wins",	"Losses",	"Draws",	"Real Win %"]
      4 )

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:319, in SparkAI.create_df(self, desc, columns, cache)
    317 if is_url:
    318     desc = soup.title.string
--> 319 return self._create_dataframe_with_llm(
    320     page_content, desc, columns, cache
    321 )

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:223, in SparkAI._create_dataframe_with_llm(self, text, desc, columns, cache)
    220 formatted_sql_query = CodeLogger.colorize_code(sql_query, "sql")
    221 self.l

In [0]:
display(df_40K_rates)

answer
"Warhammer Stats Administratum TTBattles App 40k Stats Goonhammer Media The UTC Buttscribe Core Games Other Games Columns Gaming Tactics Reviews Hobby Site News Win Rate Win Rate and Points by Faction Win Rate by Faction/Mission Win Rate by Detachment Win Rate: Faction vs Faction Scoring Primary Scoring (All) Primary Scoring (Faction) Secondary Scoring (All) Secondary Scoring (Faction) Fixed vs Tactical Scoring Visualizations Faction Popularity Events Event Placings and Lists Top Finishes by Faction TiWP Report Faction Pages Mission Pack: Start Date: End Date: Follow us Facebook Instagram Twitter YouTube Are you a Goonhammer Patron? Click here to disable ads Manage your cookie settings 40kstats.com is a presentation of Goonhammer. Copyright Goonhammer, LLC Title Close"


In [0]:
# an interesting failure case where the data is ingested but fails to be converted to a dataframe, possibly due to a character limit
df_pop = spark_ai.create_df(
  "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population", 
  columns=["Country / Dependency", "Population",	"% of world",	"Date"	"Source (official or from the United Nations)"]
)

INFO: Parsing URL: https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

INFO: SQL query for the ingestion:
CREATE OR REPLACE TEMP VIEW country_population AS 
SELECT 
    "Country / Dependency" AS country,
    "Population" AS population,
    "% ofworld" AS percentage_of_world,
    "Date Source (official or from the United Nations)" AS data_source
FROM 
    (VALUES 
        ('China', 1411750000, 17.5, '31 Dec 2022 Official estimate'),
        ('India', 1392329000, 17.3, '1 Mar 2023 Official projection'),
        ('United States', 335027000, 4.2, '7 Jul 2023 National population clock'),
        ('Indonesia', 277749853, 3.5, '31 Dec 2022 Official estimate'),
        ('Pakistan', 220425254, 2.7, '1 Jul 2020 Official projection'),
        ('Nigeria', 216783400, 2.7, '21 Mar 2022 Official projection'),
        ('Brazil', 203062512, 2.5, '1 Aug 2022 2022 census result'),
        ('Bangladesh', 169828911, 2.1, '15 Jun 2022 2022 census result'),
        ('Russia', 1464

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-3312646010236181>:1
----> 1 df_gdp = spark_ai.create_df(
      2   "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population", 
      3   columns=["Country / Dependency", "Population",	"% of world",	"Date"	"Source (official or from the United Nations)"]
      4 )

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:319, in SparkAI.create_df(self, desc, columns, cache)
    317 if is_url:
    318     desc = soup.title.string
--> 319 return self._create_dataframe_with_llm(
    320     page_content, desc, columns, cache
    321 )

File /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/pyspark_ai/pyspark_ai.py:225, in SparkAI._create_dataframe_with_llm(self, text, desc, columns, cache)
    223 view_name = self._extr